In [ ]:
import os
import json
import subprocess

In [ ]:
#整理文本为bert适合的格式
def process_text_for_bert(record_path = "//10.200.42.124/videos/segments",
                    ori_path = "data/text",
                    save_path = "data/bert/samples"):
    '''
    将精彩片段抽取出来，正例反例分开
    :param record_path:
    :param ori_path:
    :param save_path:
    :return:
    '''
    #with open("data/stopwords.txt",encoding='utf8') as f:
        #stop_words=set(f.read().split('\n'))
    stop_words = ['囡', '嗯', '吖', '唉', '呀']

    count=0
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    pf=open(os.path.join(save_path,'positive.txt'),'w',encoding='utf8')
    nf=open(os.path.join(save_path,'negative.txt'),'w',encoding='utf8')
    for date_path in os.listdir(ori_path):
        for text_path in os.listdir(os.path.join(ori_path,date_path)):
            print("processing... ",ori_path,date_path,text_path)
            highlights = set()
            # if exists highlights
            if os.path.exists(os.path.join(record_path,date_path,text_path[:-4],'pos','p')):
                good_frames=os.listdir(os.path.join(record_path,date_path,text_path[:-4],'pos','p'))
                if not good_frames:
                    print("no highlights")
                    continue
                for frame in good_frames:
                    try:
                        minute=((int(frame.split('.')[0])-3)*10+9)//60
                    except Exception as e:
                        print(e)
                        print(frame)
                    else:
                        if minute<0:
                            minute=0
                        highlights.add(minute)
            # no highlights
            else:
                print("no highlights")
                continue
            print("highlights: ",highlights)
            count+=1
            with open(os.path.join(ori_path,date_path,text_path),encoding='utf8') as f:
                for i,line in enumerate(f):
                    line=line.strip()
                    if not line: # empty line (minute)
                        continue
                    #line=line.replace(' ','。')
                    for sw in stop_words:
                        line=line.replace(sw,'')
                    
                    if i in highlights:
                        pf.write(line+'\n')
                    else:
                        nf.write(line+'\n')
    pf.close()
    nf.close()
    print("number of valid files:",count)
    return 0

In [ ]:
process_text_for_bert()

In [ ]:
#提取正样本特征
bert_path="C:\\Users\\v_sunhao7\\PycharmProjects\\bert"
bert_model_path="C:\\Users\\v_sunhao7\\PycharmProjects\\bert\\chinese_L-12_H-768_A-12"
output_path="data/bert/parameter/"
input_path="data/bert/samples/"
if not os.path.exists(output_path):
    os.makedirs(output_path)
cmd="python %s/extract_features.py \
  --input_file=%s \
  --output_file=%s \
  --vocab_file=%s/vocab.txt \
  --bert_config_file=%s/bert_config.json \
  --init_checkpoint=%s/bert_model.ckpt \
  --layers=-1 \
  --max_seq_length=512 \
  --batch_size=8"%(bert_path, input_path+"positive.txt", output_path+"positive.json", bert_model_path, bert_model_path, bert_model_path)

subprocess.call(cmd)

In [ ]:
#提取负样本特征
bert_path="C:\\Users\\v_sunhao7\\PycharmProjects\\bert"
bert_model_path="C:\\Users\\v_sunhao7\\PycharmProjects\\bert\\chinese_L-12_H-768_A-12"
output_path="data/bert/parameter/"
input_path="data/bert/samples/"
if not os.path.exists(output_path):
    os.makedirs(output_path)
cmd="python %s/extract_features.py \
  --input_file=%s \
  --output_file=%s \
  --vocab_file=%s/vocab.txt \
  --bert_config_file=%s/bert_config.json \
  --init_checkpoint=%s/bert_model.ckpt \
  --layers=-1 \
  --max_seq_length=512 \
  --batch_size=8"%(bert_path, input_path+"negative.txt", output_path+"negative.json", bert_model_path, bert_model_path, bert_model_path)

subprocess.call(cmd)

In [ ]:
import numpy as np
#整理成矩阵并保存

pos_matrix=np.zeros((0,768))
with open("data/bert/parameter/positive.json", encoding='utf8') as f:
    for line in f:
        t=json.loads(line)
        pos_matrix=np.vstack((pos_matrix,np.array(t['features'][0]['layers'][0]['values'])))
        
neg_matrix=np.zeros((0,768))
with open("data/bert/parameter/negative.json", encoding='utf8') as f:
    for line in f:
        t=json.loads(line)
        neg_matrix=np.vstack((neg_matrix,np.array(t['features'][0]['layers'][0]['values'])))
        
np.save('data/bert/matrix/pos_matrix.npy',pos_matrix)
np.save('data/bert/matrix/neg_matrix.npy',neg_matrix)

## pytorch做分类

In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torch.utils.data


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #self.fc1 = nn.Linear(768, 32)
        self.fc1 = nn.Linear(768, 1)

    def forward(self, x):
        #x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc1(x))
        return x
    
net = Net()


criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
#load data
pos_matrix=np.load("data/bert/matrix/pos_matrix.npy")
neg_matrix=np.load("data/bert/matrix/neg_matrix.npy")
#shuffle
np.random.shuffle(pos_matrix)
np.random.shuffle(neg_matrix)
#split data
train_prop=0.7
repeats=4  #训练集中，正例的复制倍数
pos_train_size=int(pos_matrix.shape[0]*train_prop)
neg_train_size=int(neg_matrix.shape[0]*train_prop)

train_pos=pos_matrix[:pos_train_size]
train_neg=neg_matrix[:neg_train_size]
train_pos=np.tile(train_pos,(repeats,1))
test_pos=pos_matrix[pos_train_size:]
test_neg=neg_matrix[neg_train_size:]
#test_pos=np.tile(test_pos,(6,1))

x_train=np.vstack((train_pos,train_neg))
y_train=np.vstack((np.ones((len(train_pos),1)),np.zeros((len(train_neg),1))))
x_test=np.vstack((test_pos,test_neg))
y_test=np.vstack((np.ones((len(test_pos),1)),np.zeros((len(test_neg),1))))

#convert to pytorch tensor
x_train=torch.from_numpy(x_train)
x_test=torch.from_numpy(x_test)
y_train=torch.from_numpy(y_train)
y_test=torch.from_numpy(y_test)
#data loader
trainset=torch.utils.data.TensorDataset(x_train, y_train)
testset=torch.utils.data.TensorDataset(x_test, y_test)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100,
                                          shuffle=True, num_workers=2)

In [ ]:
len(test_neg)/len(test_pos)

In [ ]:
for epoch in range(100):  # loop over the dataset multiple times
    print("training...... epoch:", epoch)
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs = inputs.float()
        labels = labels.float()
        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
    print("loss:", running_loss/i)

print('Finished Training')

In [ ]:
#train f score
precision=torch.sum(torch.round(net(Variable(x_train[:].float())))*y_train.float())/torch.sum(torch.round(net(Variable(x_train[:].float()))))
recall=torch.sum(torch.round(net(Variable(x_train[:].float())))*y_train.float())/torch.sum(y_train.float())
f1=precision*recall*2/(precision+recall)
print(precision,recall,f1)

In [ ]:
# test f score
precision=torch.sum(torch.round(net(Variable(x_test[:].float())))*y_test.float())/torch.sum(torch.round(net(Variable(x_test[:].float()))))
recall=torch.sum(torch.round(net(Variable(x_test[:].float())))*y_test.float())/torch.sum(y_test.float())
f1=precision*recall*2/(precision+recall)
print(precision,recall,f1)

In [ ]:
torch.save(net,"model/bert.output.pkl")

In [ ]:
net.state_dict()